In [1]:
import requests
from bs4 import BeautifulSoup
import re
import warnings
import json
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [ ]:
path = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
base = 'https://www.imdb.com'
headers = {'User-Agent': 'Safari/605.1.15'}

# GET LINKs

In [3]:
r = requests.get(f'{path}')
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('tbody', class_ = 'lister-list')
movies = s.find_all('td',class_ = 'titleColumn')
links = []
for movie in movies:
    link = movie.a['href']
    links.append(f'{link}')

save links

In [4]:
links = np.array(links)
np.save('top_250_Linkes',links)

load links

In [10]:
links = np.load('top_250_Linkes.npy')

# GET DATA

In [ ]:
xrror_gross_us_canda = []
xrror_runtime = []
xrror_genre_movie = []
xrror_cast = []
xrror_cast = []
xrror_storyline = []
xrror = []

movies = pd.DataFrame(columns = ['id','title','year','runtime','parental_guide','gross_us_canda'])
genre_movie =  pd.DataFrame(columns = ['movie_id','genre'])
cast = pd.DataFrame(columns = ['movie_id','person_id'])
crew = pd.DataFrame(columns = ['movie_id','person_id','role'])
person = pd.DataFrame(columns = ['person_id','name'])
storyline = pd.DataFrame(columns = ['movie_id','content'])

In [129]:
for link in links:
    
    r = requests.get(f'{base}{link}', headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    try:
        movie_id = link[9:-1]
        try:
            content = soup.find('div', class_ = 'ipc-html-content-inner-div').text
            storyline.loc[len(storyline.index)] = [movie_id,content]

        except Exception as e:
            storyline.loc[len(storyline.index)] = [movie_id,'']
            print(f'storyline {e}')
            xrror_storyline.append(link)
        title = soup.find('span', class_ = 'sc-afe43def-1 fDTGTb').text
        info = soup.find_all('li',role ='presentation',class_="ipc-inline-list__item")
        year = int(info[3].text)
        parental_guide = info[4].text
        runtime = info[5].text
        try :
            runtime = int(runtime[0])*60 + int(runtime[3:-1])
        except Exception as e: 
            print(f'runtime {e}')       
            xrror_runtime.append(link)
        try:
            gross_us_canda = soup.find_all('span', class_ = 'ipc-metadata-list-item__list-content-item')[3].text
        except Exception as e:
            gross_us_canda = ''
            print(f'gross_us_canda {e}')       
            xrror_gross_us_canda.append(link)
        movies.loc[len(movies.index)] = [movie_id,title,year,runtime,parental_guide,gross_us_canda]
        genres = soup.find_all('a', class_ = 'ipc-chip ipc-chip--on-baseAlt')
        for g in genres:
            try: 
                genre_movie.loc[len(genre_movie.index)] = [movie_id,g.text]
            except Exception as e: 
                print(f'genre_movie {e}')       
                xrror_genre_movie.append(link)
        for i in soup.find_all('li', 
            class_ = 'ipc-metadata-list__item')[:3]:
            span = i.find_all('a'
                ,class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
            role = f'{i.text[:8]}'
            for s in span:
                name = s.text
                p_link = s['href']
                person.loc[len(person.index)] = [p_link[8:-15].zfill(7),name]
                person.drop_duplicates(inplace = True)
                if (role[:5] == 'Stars'):
                    try:
                        cast.loc[len(cast.index)] = [movie_id,p_link[8:-15].zfill(7)]
                    except Exception as e: 
                        print(f'cast {e}')       
                        xrror_cast.append(link)
                else:
                    try:
                        crew.loc[len(crew.index)] = [movie_id,p_link[8:-15].zfill(7),role[:-2]]
                    except Exception as e: 
                        print(f'crew {e}')       
                        xrror_crew.append(link)
                        
    except Exception as e: 
        print(e)       
        xrror.append(link)

Handle Error HARDCODE :|

In [ ]:
runtime = movies[movies['id'] == '0082096'].parental_guide.iloc[0]
false_value = movies[movies['id'] == '0082096'].runtime.iloc[0]
movies['parental_guide'].replace(runtime,'',inplace = True)
movies['runtime'].replace(false_value, int(runtime[0])*60 + int(runtime[3:-1]),inplace = True)

movies['runtime'].replace('2h',120,inplace = True)
movies['runtime'].replace('3h',180,inplace = True)
movies['runtime'].replace('4h',240,inplace = True)

save links data 

In [2]:
cast.to_csv('cast.csv',index=False)
crew.to_csv('crew.csv',index=False)
person.to_csv('person.csv',index=False)
genre_movie.to_csv('genre_movie.csv',index=False)
movies.to_csv('movies.csv',index=False)
storyline.to_csv('storyline.csv',index=False)

NameError: name 'cast' is not defined

# LOAD CSVs 

In [69]:
movies = pd.read_csv('movies.csv',dtype=object)
genre_movie =  pd.read_csv('genre_movie.csv',dtype=object)
cast = pd.read_csv('cast.csv',dtype=object)
crew = pd.read_csv('crew.csv',dtype=object)
person = pd.read_csv('person.csv',dtype=object)
storyline = pd.read_csv('storyline.csv',dtype=object)

Fix Writers

In [70]:
crew.loc[crew['role'].str.startswith('Writer', na=False) , 'role'] = 'Writer'
crew.to_csv('crew.csv',index=False)

# fix csv

In [71]:
movies

,id,title,year,runtime,parental_guide,gross_us_canda
0,0111161,The Shawshank Redemption,1994,142,R,"$28,767,189"
1,0068646,The Godfather,1972,175,R,"$136,381,073"
2,0468569,The Dark Knight,2008,152,PG-13,"$534,987,076"
3,0071562,The Godfather Part II,1974,202,R,"$47,834,595"
4,0050083,12 Angry Men,1957,96,Approved,$955
...,...,...,...,...,...,...
245,0129167,The Iron Giant,1999,86,PG,"$23,315,035"
246,1454029,The Help,2011,146,PG-13,"$169,708,112"
247,0103639,Aladdin,1992,90,G,"$217,350,219"
248,0071411,Dersu Uzala,1975,142,G,"$14,480"


In [72]:
# replace gross_us_canda where $ is not present with nan
movies.loc[~(movies['gross_us_canda'].str.find('$') != -1),'gross_us_canda'] = np.nan

## remove $ and , from gross_us_canda
movies['gross_us_canda'] = movies['gross_us_canda'].str.replace('$','')
movies['gross_us_canda'] = movies['gross_us_canda'].str.replace(',','')

# change gross_us_canda to float
movies['gross_us_canda'] = movies['gross_us_canda'].astype(float)

## replace not rated and unrated with unrated
movies['parental_guide'] = movies['parental_guide'].replace('Not Rated','Unrated')

# remove m frome runtime
movies['runtime'] = movies['runtime'].str.replace('m','')

# change dtype year to int, runtime to int, parental_guide 
movies['year'] = movies['year'].astype(float)
movies['runtime'] = movies['runtime'].astype(float)

# remove 0 from start of id
movies['id'] = movies['id'].str.replace('^0+','')

## rename title to Title, year to Year, runtime to Runtime, parental_guide to Parental_Guide, gross_us_canda to Gross_US_Canda
movies.rename(columns={'title':'Title',
                       'year':'Year',
                       'runtime':'Runtime',
                       'parental_guide':'Parental_Guide',
                       'gross_us_canda':'Gross_US_Canda'},inplace=True)

movies

,id,Title,Year,Runtime,Parental_Guide,Gross_US_Canda
0,111161,The Shawshank Redemption,1994.0,142.0,R,28767189.0
1,68646,The Godfather,1972.0,175.0,R,136381073.0
2,468569,The Dark Knight,2008.0,152.0,PG-13,534987076.0
3,71562,The Godfather Part II,1974.0,202.0,R,47834595.0
4,50083,12 Angry Men,1957.0,96.0,Approved,955.0
...,...,...,...,...,...,...
245,129167,The Iron Giant,1999.0,86.0,PG,23315035.0
246,1454029,The Help,2011.0,146.0,PG-13,169708112.0
247,103639,Aladdin,1992.0,90.0,G,217350219.0
248,71411,Dersu Uzala,1975.0,142.0,G,14480.0


In [73]:
movies["id"].duplicated().sum()

0

In [74]:
movies["Parental_Guide"].value_counts()

R           98
PG          36
PG-13       35
Unrated     25
G           19
Passed      16
Approved    14
18+          2
TV-PG        1
X            1
TV-MA        1
GP           1
Name: Parental_Guide, dtype: int64

## person

In [75]:
person

,person_id,name
0,0001104,Frank Darabont
1,0000175,Stephen King
2,0000209,Tim Robbins
3,0000151,Morgan Freeman
4,0348409,Bob Gunton
...,...,...
1070,0125730,Mikhail Bychkov
1071,0000126,Kevin Costner
1072,0086658,Michael Blake
1073,0001521,Mary McDonnell


In [76]:
# rename person_id to id
person.rename(columns={'person_id':'id'}, inplace=True)

# remove 0 from start of id
person['id'] = person['id'].str.replace('^0+','')

## rename name to Name
person.rename(columns={'name':'Name'}, inplace=True)

person["id"].duplicated().sum()
person


,id,Name
0,1104,Frank Darabont
1,175,Stephen King
2,209,Tim Robbins
3,151,Morgan Freeman
4,348409,Bob Gunton
...,...,...
1070,125730,Mikhail Bychkov
1071,126,Kevin Costner
1072,86658,Michael Blake
1073,1521,Mary McDonnell


## genre_movie

In [77]:
genre_movie

,movie_id,genre
0,0111161,Drama
1,0068646,Crime
2,0068646,Drama
3,0468569,Action
4,0468569,Crime
...,...,...
622,0071411,Biography
623,0071411,Drama
624,0099348,Adventure
625,0099348,Drama


In [78]:
## remove 0 from start of movie_id
genre_movie['movie_id'] = genre_movie['movie_id'].str.replace('^0+','')

## rename movie_id to Movie_id, genre to Genre
genre_movie.rename(columns={'movie_id':'Movie_id',
                            'genre':'Genre'}, inplace=True)
genre_movie

,Movie_id,Genre
0,111161,Drama
1,68646,Crime
2,68646,Drama
3,468569,Action
4,468569,Crime
...,...,...
622,71411,Biography
623,71411,Drama
624,99348,Adventure
625,99348,Drama


## cast

In [79]:
cast

,movie_id,person_id
0,0111161,0000209
1,0111161,0000151
2,0111161,0348409
3,0068646,0000008
4,0068646,0000199
...,...,...
1495,0099348,0813272
1496,0099348,0125730
1497,0099348,0000126
1498,0099348,0001521


In [80]:
## remove 0 from start of movie_id
cast['movie_id'] = cast['movie_id'].str.replace('^0+','')

## remove 0 from start of person_id
cast['person_id'] = cast['person_id'].str.replace('^0+','')

## rename movie_id to Movie_id, person_id to Person_id
cast.rename(columns={'movie_id':'Movie_id',
                     'person_id':'Person_id'}, inplace=True)
cast

,Movie_id,Person_id
0,111161,209
1,111161,151
2,111161,348409
3,68646,8
4,68646,199
...,...,...
1495,99348,813272
1496,99348,125730
1497,99348,126
1498,99348,1521


## crew

In [81]:
crew

,movie_id,person_id,role
0,0111161,0001104,Director
1,0111161,0000175,Writer
2,0111161,0001104,Writer
3,0068646,0000338,Director
4,0068646,0701374,Writer
...,...,...,...
1663,0099348,0000041,Writer
1664,0099348,0619330,Writer
1665,0099348,0037539,Writer
1666,0099348,0000126,Director


In [82]:
## remove 0 from start of movie_id
crew['movie_id'] = crew['movie_id'].str.replace('^0+','')

## remove 0 from start of person_id
crew['person_id'] = crew['person_id'].str.replace('^0+','')

## rename movie_id to Movie_id, person_id to Person_id, role to Role
crew.rename(columns={'movie_id':'Movie_id',
                     'person_id':'Person_id',
                     'role':'Role'}, inplace=True)
crew

,Movie_id,Person_id,Role
0,111161,1104,Director
1,111161,175,Writer
2,111161,1104,Writer
3,68646,338,Director
4,68646,701374,Writer
...,...,...,...
1663,99348,41,Writer
1664,99348,619330,Writer
1665,99348,37539,Writer
1666,99348,126,Director


## storyline

In [83]:
storyline

,movie_id,content
0,0111161,The Shawshank Redemption has become a classic ...
1,0068646,Cinematographer Gordon Willis earned himself t...
2,0468569,"In Sir Michael Caine's opinion, Heath Ledger b..."
3,0071562,Robert De Niro spent four months learning to s...
4,0050083,Director Sidney Lumet had the actors all stay ...
...,...,...
245,0129167,The scene where the Giant's hand was in the li...
246,1454029,"Kathryn Stockett's book, on which this film is..."
247,0103639,"Originally, Jafar was more hot-tempered while ..."
248,0071411,Akira Kurosawa had hoped to make this film as ...


In [84]:
## remove 0 from start of movie_id
storyline['movie_id'] = storyline['movie_id'].str.replace('^0+','')

## rename movie_id to Movie_id, storyline to Storyline
storyline.rename(columns={'movie_id':'Movie_id',
                          'storyline':'Storyline'}, inplace=True)

storyline

,Movie_id,content
0,111161,The Shawshank Redemption has become a classic ...
1,68646,Cinematographer Gordon Willis earned himself t...
2,468569,"In Sir Michael Caine's opinion, Heath Ledger b..."
3,71562,Robert De Niro spent four months learning to s...
4,50083,Director Sidney Lumet had the actors all stay ...
...,...,...
245,129167,The scene where the Giant's hand was in the li...
246,1454029,"Kathryn Stockett's book, on which this film is..."
247,103639,"Originally, Jafar was more hot-tempered while ..."
248,71411,Akira Kurosawa had hoped to make this film as ...
